# Stochastic Volume Calculations

OpenMC has a capability to stochastically determine volumes of cells, materials, and universes. The method works by overlaying a bounding box, sampling points from within the box, and seeing what fraction of points were found in a desired domain. The benefit of doing this stochastically (as opposed to equally-spaced points), is that it allows us to give an error estimate on each stochastic quantity.

In this example, we will use the stochastic volume capability in OpenMC to calculation microscopic cross sections on a heterogeneous cell.

In [ ]:
from math import pi
import openmc

import sys
sys.path.append('..')
from inputs import *

First let's start by creating our materials and geometry:

In [ ]:
fuel = openmc.Material(1)
fuel.add_element('U', 1, enrichment=3.0)
fuel.add_element('O', 2)
fuel.set_density('g/cm3', 10.0)

water = openmc.Material(2)
water.add_nuclide('H1', 2)
water.add_nuclide('O16', 1)
water.add_s_alpha_beta('c_H_in_H2O')
water.set_density('g/cm3', 1)

mats = openmc.Materials([fuel, water])
mats.export_to_xml()

In [ ]:
s1 = openmc.Sphere(R=10.0, boundary_type='vacuum')
s2 = openmc.Sphere(R=5.0)

inner_sphere = openmc.Cell(1, fill=fuel, region=-s2)
outer_sphere = openmc.Cell(2, fill=water, region=+s2 & -s1)
root = openmc.Universe(0, cells=(inner_sphere, outer_sphere))

geom = openmc.Geometry(root)
geom.export_to_xml()

The `VolumeCalculation` object accepts four arguments: the domains to find volumes of, how many samples you want, the lower-left coordinates of a bounding box, and the upper-right coordinates of a bounding box. For many cells, the bounding-box can be determined automatically so there is no need to specify the lower-left and upper-right coordinates in such cases.